### Veri Okuma

Verimiz yaklaşık 13 milyon satır 5 sütundan oluştuğu için süreci pandas yerine polars kütüphanesi ile devam edeceğiz.

Polars pandas kütüphanesin göre çok daha hızlı veri okuma hızına sahiptir.

In [2]:
import polars as pl

veri=pl.read_csv("hamveri.csv")
print(veri)

shape: (13_608_260, 5)
┌──────────────────┬────────────┬─────────────────────┬───────────────────────────┬─────────────┐
│ tarihSaat        ┆ otobusTuru ┆ biletGrubu          ┆ hat                       ┆ toplamBinis │
│ ---              ┆ ---        ┆ ---                 ┆ ---                       ┆ ---         │
│ str              ┆ str        ┆ str                 ┆ str                       ┆ i64         │
╞══════════════════╪════════════╪═════════════════════╪═══════════════════════════╪═════════════╡
│ 01/01/2021 00:00 ┆ EGO        ┆ TAM KART            ┆ 540 - ERYAMAN - SIHHİYE   ┆ 1           │
│ 01/01/2021 00:00 ┆ EGO        ┆ TAM KART            ┆ 1832 - ULUS - İLKER SİNAN ┆ 1           │
│                  ┆            ┆                     ┆ CADD…                     ┆             │
│ 01/01/2021 00:00 ┆ EGO        ┆ TAM KART            ┆ 523 - SİNCAN - POLATLI 2. ┆ 1           │
│                  ┆            ┆                     ┆ CAD.…                     ┆            

### Eksik Gözlem Kontrol

In [3]:
print(veri.select(pl.all().null_count()))

shape: (1, 5)
┌───────────┬────────────┬────────────┬─────┬─────────────┐
│ tarihSaat ┆ otobusTuru ┆ biletGrubu ┆ hat ┆ toplamBinis │
│ ---       ┆ ---        ┆ ---        ┆ --- ┆ ---         │
│ u32       ┆ u32        ┆ u32        ┆ u32 ┆ u32         │
╞═══════════╪════════════╪════════════╪═════╪═════════════╡
│ 0         ┆ 0          ┆ 0          ┆ 0   ┆ 0           │
└───────────┴────────────┴────────────┴─────┴─────────────┘


###

Tarih ve saat str cinsinde olduğu için tarih ve saat ayrımı yapamayız.
Bu yüzden önce zaman formatına çevireceğiz ve daha sonra ayıracağız.

In [4]:
veri=veri.with_columns(
    pl.col("tarihSaat").str.to_datetime().alias("tarihSaat"))
    
veri=veri.with_columns([
    pl.col("tarihSaat").dt.strftime("%d-%m-%Y").alias("tarih"),
    pl.col("tarihSaat").dt.strftime("%H:%M").alias("saat")])
 
veri=veri.drop("tarihSaat")
print(veri)

shape: (13_608_260, 6)
┌────────────┬─────────────────────┬────────────────────────────┬─────────────┬────────────┬───────┐
│ otobusTuru ┆ biletGrubu          ┆ hat                        ┆ toplamBinis ┆ tarih      ┆ saat  │
│ ---        ┆ ---                 ┆ ---                        ┆ ---         ┆ ---        ┆ ---   │
│ str        ┆ str                 ┆ str                        ┆ i64         ┆ str        ┆ str   │
╞════════════╪═════════════════════╪════════════════════════════╪═════════════╪════════════╪═══════╡
│ EGO        ┆ TAM KART            ┆ 540 - ERYAMAN - SIHHİYE    ┆ 1           ┆ 01-01-2021 ┆ 00:00 │
│ EGO        ┆ TAM KART            ┆ 1832 - ULUS - İLKER SİNAN  ┆ 1           ┆ 01-01-2021 ┆ 00:00 │
│            ┆                     ┆ CADD…                      ┆             ┆            ┆       │
│ EGO        ┆ TAM KART            ┆ 523 - SİNCAN - POLATLI 2.  ┆ 1           ┆ 01-01-2021 ┆ 00:00 │
│            ┆                     ┆ CAD.…                      ┆   

###
Hat numarası ve hat isimleri aynı sütunda. Bunları ayıralım.

In [5]:
veri=veri.with_columns([
    pl.col("hat").str.split("-").list.get(0).alias("hat_no"), 
    pl.col("hat").str.split("-").list.slice(1).list.join("-").alias("hat_adı")])

veri=veri.drop("hat")
print(veri)


shape: (13_608_260, 7)
┌────────────┬────────────────────┬─────────────┬────────────┬───────┬────────┬────────────────────┐
│ otobusTuru ┆ biletGrubu         ┆ toplamBinis ┆ tarih      ┆ saat  ┆ hat_no ┆ hat_adı            │
│ ---        ┆ ---                ┆ ---         ┆ ---        ┆ ---   ┆ ---    ┆ ---                │
│ str        ┆ str                ┆ i64         ┆ str        ┆ str   ┆ str    ┆ str                │
╞════════════╪════════════════════╪═════════════╪════════════╪═══════╪════════╪════════════════════╡
│ EGO        ┆ TAM KART           ┆ 1           ┆ 01-01-2021 ┆ 00:00 ┆ 540    ┆  ERYAMAN - SIHHİYE │
│ EGO        ┆ TAM KART           ┆ 1           ┆ 01-01-2021 ┆ 00:00 ┆ 1832   ┆ ULUS - İLKER SİNAN │
│            ┆                    ┆             ┆            ┆       ┆        ┆ CADDE              │
│ EGO        ┆ TAM KART           ┆ 1           ┆ 01-01-2021 ┆ 00:00 ┆ 523    ┆ SİNCAN - POLATLI   │
│            ┆                    ┆             ┆            ┆      

### 
Ankara'da hat numaraları 3 hanelidir. 3 den fazla hane var ise hattın farklı türleridir ve "-" ile ayrılır.
Örneğin 5106 hat numarası 510-6 şeklindedir. Ancak veri setimizde 4 hane geldiği için bunları düzeltmemiz lazım.

In [6]:
veri=veri.with_columns(
    pl.col("hat_no").str.strip_chars().alias("hat_no"))

veri=veri.with_columns(
    pl.col("hat_no").map_elements(lambda x: f"{x[:3]}-{x[3:]}" if len(x)>3 else x,
                                  return_dtype=pl.Utf8).alias("hat_no"))

print(veri)

shape: (13_608_260, 7)
┌────────────┬────────────────────┬─────────────┬────────────┬───────┬────────┬────────────────────┐
│ otobusTuru ┆ biletGrubu         ┆ toplamBinis ┆ tarih      ┆ saat  ┆ hat_no ┆ hat_adı            │
│ ---        ┆ ---                ┆ ---         ┆ ---        ┆ ---   ┆ ---    ┆ ---                │
│ str        ┆ str                ┆ i64         ┆ str        ┆ str   ┆ str    ┆ str                │
╞════════════╪════════════════════╪═════════════╪════════════╪═══════╪════════╪════════════════════╡
│ EGO        ┆ TAM KART           ┆ 1           ┆ 01-01-2021 ┆ 00:00 ┆ 540    ┆  ERYAMAN - SIHHİYE │
│ EGO        ┆ TAM KART           ┆ 1           ┆ 01-01-2021 ┆ 00:00 ┆ 183-2  ┆ ULUS - İLKER SİNAN │
│            ┆                    ┆             ┆            ┆       ┆        ┆ CADDE              │
│ EGO        ┆ TAM KART           ┆ 1           ┆ 01-01-2021 ┆ 00:00 ┆ 523    ┆ SİNCAN - POLATLI   │
│            ┆                    ┆             ┆            ┆      

### 
Hat verilerini kullanarak süre ve mesafeyi de ekleyelim.

In [7]:
hatveri=pl.read_csv("hatbilgiveri.csv")

veri=veri.join(hatveri,on="hat_no",how="left")
print(veri)

shape: (13_608_260, 9)
┌────────────┬────────────┬────────────┬────────────┬───┬────────┬────────────┬────────┬───────────┐
│ otobusTuru ┆ biletGrubu ┆ toplamBini ┆ tarih      ┆ … ┆ hat_no ┆ hat_adı    ┆ Mesafe ┆ Süre      │
│ ---        ┆ ---        ┆ s          ┆ ---        ┆   ┆ ---    ┆ ---        ┆ (Km)   ┆ (Dakika)  │
│ str        ┆ str        ┆ ---        ┆ str        ┆   ┆ str    ┆ str        ┆ ---    ┆ ---       │
│            ┆            ┆ i64        ┆            ┆   ┆        ┆            ┆ i64    ┆ i64       │
╞════════════╪════════════╪════════════╪════════════╪═══╪════════╪════════════╪════════╪═══════════╡
│ EGO        ┆ TAM KART   ┆ 1          ┆ 01-01-2021 ┆ … ┆ 540    ┆ ERYAMAN -  ┆ 62     ┆ 105       │
│            ┆            ┆            ┆            ┆   ┆        ┆ SIHHİYE    ┆        ┆           │
│ EGO        ┆ TAM KART   ┆ 1          ┆ 01-01-2021 ┆ … ┆ 183-2  ┆ ULUS -     ┆ 15     ┆ 45        │
│            ┆            ┆            ┆            ┆   ┆        ┆ İ

###

Sütun isimlerini değiştirelim

In [8]:
veri=(
    veri.rename({
        "otobusTuru":"Araç Türü",
        "biletGrubu":"Bilet Türü",
        "toplamBinis":"Biniş Adeti",
        "tarih":"Tarih",
        "saat":"Saat",
        "hat_no":"Hat No",
        "hat_adı":"Hat Adı"    
    })
    .select([
        "Tarih","Saat","Hat No","Hat Adı",
        "Araç Türü","Bilet Türü","Biniş Adeti",
        "Süre (Dakika)",
        "Mesafe (Km)"]))
print(veri)

shape: (13_608_260, 9)
┌────────────┬───────┬────────┬───────────────┬───┬──────────────┬───────┬──────────┬──────────────┐
│ Tarih      ┆ Saat  ┆ Hat No ┆ Hat Adı       ┆ … ┆ Bilet Türü   ┆ Biniş ┆ Süre     ┆ Mesafe (Km)  │
│ ---        ┆ ---   ┆ ---    ┆ ---           ┆   ┆ ---          ┆ Adeti ┆ (Dakika) ┆ ---          │
│ str        ┆ str   ┆ str    ┆ str           ┆   ┆ str          ┆ ---   ┆ ---      ┆ i64          │
│            ┆       ┆        ┆               ┆   ┆              ┆ i64   ┆ i64      ┆              │
╞════════════╪═══════╪════════╪═══════════════╪═══╪══════════════╪═══════╪══════════╪══════════════╡
│ 01-01-2021 ┆ 00:00 ┆ 540    ┆ ERYAMAN -     ┆ … ┆ TAM KART     ┆ 1     ┆ 105      ┆ 62           │
│            ┆       ┆        ┆ SIHHİYE       ┆   ┆              ┆       ┆          ┆              │
│ 01-01-2021 ┆ 00:00 ┆ 183-2  ┆ ULUS - İLKER  ┆ … ┆ TAM KART     ┆ 1     ┆ 45       ┆ 15           │
│            ┆       ┆        ┆ SİNAN CADDE   ┆   ┆              ┆  

###
Verimiz artık nihai verisetimiz olarak kaydedelim.

In [9]:

veri.write_csv("nihaiveri.csv")